# 1. Preparations

In [0]:
#@markdown __<-- Run first__ *click the play button to run cell*

#@markdown - Install libraries
#@markdown - Import libraries (e.g. Basic, Visualization, Unsupervised Learning)
#@markdown - Helper Code
%%capture
# Basic Libraries
import os
os.system('pip install wget pivottablejs shap category_encoders pyclustering')
import wget, pandas_profiling, uuid, shap
from google.colab import widgets, output, data_table, files
from IPython.display import clear_output, HTML, display
import ipywidgets
import pandas as pd
import numpy as np
import gc
import psutil
from joblib import Memory
location = '/content'
memory = Memory(location, verbose=0)
import warnings
from pprint import pprint
warnings.filterwarnings("ignore")

# Visualization Libraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

# ML Libraries
from category_encoders import BinaryEncoder, OrdinalEncoder
from sklearn import preprocessing, metrics, svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score, davies_bouldin_score, calinski_harabasz_score, fowlkes_mallows_score, homogeneity_score, mutual_info_score, v_measure_score
from sklearn.cluster import AgglomerativeClustering, SpectralClustering, DBSCAN, OPTICS
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.cluster.kmeans import kmeans
from pyclustering.cluster.elbow import elbow


class SelectOption(object):
  def __init__(self, title, callback, options):
    self._title = title
    self._callback = callback
    self.options = ''' 
'''.join([f'<option value= "{i}"> {i}'for i in options])

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """
    {title}
    <select id={callback_id}>
"""+self.options+"""
    </select>
        <script>
          document.querySelector("#{callback_id}").onchange = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [e.target.value], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

# button = ipywidgets.Button(description="Run!")
# output = ipywidgets.Output()

# def on_button_clicked(b):
#   with output:
#     print("Button clicked.")

# button.on_click(on_button_clicked)
# display(button, output)

Example Dataset: Give Me Some Credit Dataset
https://www.kaggle.com/c/GiveMeSomeCredit/data

In [5]:
# @title ## 1.1. Load Dataset of Your Choice {display-mode: "form" }
#@markdown __Upload File__ OR __Download from Dataset URL__
##@title Load and Preview Dataset {display-mode: "form" }
Use_file_from = "Upload" #@param ["URL", "Upload"]

def upload_file(Use_file_from):
    if Use_file_from == 'Upload':
        try:
            print(uploaded)
            if os.path.isfile(list(uploaded.keys())[0]): 
                os.remove(list(uploaded.keys())[0])
                print('Removing previous uploaded files to prevent duplicates')
        except: 
            print('')
            uploaded = files.upload()
            clear_output()

    #@markdown If you choose "Download from Dataset URL", give a URL
    if Use_file_from=='URL':
        Dataset_URL = "" #@param {type:"string"}

        dataset = wget.detect_filename(Dataset_URL)
        if not os.path.isfile(dataset): 
            os.system(f'wget -O {dataset} {Dataset_URL}')
            print(f'File {dataset} successfully downloaded from URL')
        else: 
            os.remove(dataset)
            os.system(f'wget -O {dataset} {Dataset_URL}')
            print(f'Old file removed, new file {dataset} successfully downloaded from URL')

    return uploaded

valid_upload = False

if Use_file_from == "URL":
    try:
        if os.path.isfile(dataset): 
            print('detected dataset downloaded from URL')
            valid_upload = True
    except: print('Failed URL')

    if valid_upload==False: 
        print('No file detected for URL, please try to download file again.')
    else: 
        file_ = dataset

elif Use_file_from == "Upload":
    try:
        if os.path.isfile(list(uploaded.keys())[0]): 
            print('detected dataset upload from local machine')
            valid_upload = True
    except: 
        print('No files detected. Upload files now.')
        if valid_upload==False: 
            uploaded = upload_file(Use_file_from)
            file_ = list(uploaded.keys())[0]
        else: 
            file_ = list(uploaded.keys())[0]


if '.xlsx' in file_: df = pd.read_excel(file_)
else:
    try:
        df = pd.read_csv(file_, error_bad_lines=False, delimiter=',')
    except UnicodeDecodeError:
        df = pd.read_csv(file_, encoding="ISO-8859-1", error_bad_lines=False, delimiter=',')

def detect_datetime(df):
    date_time_cols = []
    for i in df.select_dtypes(['object']).columns:
        try:
            _ = pd.to_datetime(df[i])
            date_time_cols.append(i)
        except ValueError:
            continue
    df[date_time_cols] = df[date_time_cols].apply(pd.to_datetime, axis=0)
    return df

df = detect_datetime(df)
if 'Unnamed: 0' in df.columns: df.drop('Unnamed: 0', axis=1, inplace=True)
data_table.DataTable(df, include_index=True, num_rows_per_page=10, max_rows=100)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
149995,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149996,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149997,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149998,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0


In [6]:
#@title ## 1.2. Dataset Summary {display-mode: "form" }
## @markdown <i>(Run again if output doesn't display in tabs)</i>
tb = widgets.TabBar(['Meta-Data', 'Numeric_Cols', 'DateTime_Cols', 'String_Cols'], location='top')
with tb.output_to('Meta-Data'):
#   print('''-------------------------META DATA--------------------------''')
  display(df.info())

with tb.output_to('DateTime_Cols'):
#   print('''----------------------DateTime columns-----------------------''')
  if len(df.select_dtypes(include='datetime64').columns) == 0:
    print('No DateTime column detected')
  else:
    display(df.select_dtypes(include='datetime64').head())

with tb.output_to('String_Cols'):
#   print('''----------------------String Columns-----------------------''')
  try:
    df_type_objects = pd.concat([df.select_dtypes(include='object').nunique(),df.select_dtypes(include='object').apply(lambda x: list(pd.unique(x)),axis=0)],axis=1, join='inner')
    df_type_objects.rename(columns={0:'No. Unique values',1:'Unique Values'}, inplace=True)
    display(df_type_objects)
  except:
    display(pd.DataFrame(df.select_dtypes(include='object').nunique(),columns=['No. Unique Values']))
    print('\nErrors during analysis of string columns due to extreme number of unique values')

with tb.output_to('Numeric_Cols'):
#   print('''----------------------Numeric Columns-----------------------''')
  display(df.describe().round(2))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.00,150000.00,150000.00,150000.00,150000.00,120269.00,150000.00,150000.00,150000.00,150000.00,146076.00
mean,0.07,6.05,52.30,0.42,353.01,6670.22,8.45,0.27,1.02,0.24,0.76
std,0.25,249.76,14.77,4.19,2037.82,14384.67,5.15,4.17,1.13,4.16,1.12
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.03,41.00,0.00,0.18,3400.00,5.00,0.00,0.00,0.00,0.00
50%,0.00,0.15,52.00,0.00,0.37,5400.00,8.00,0.00,1.00,0.00,0.00
75%,0.00,0.56,63.00,0.00,0.87,8249.00,11.00,0.00,2.00,0.00,1.00
max,1.00,50708.00,109.00,98.00,329664.00,3008750.00,58.00,98.00,54.00,98.00,20.00


<IPython.core.display.Javascript object>


# 2. Preprocessing
1. Replacing Null/NaN data
2. Remove Features with High Correlation
3. StandardScaling, Normalizing, PCA



In [7]:
#@title ## 2.1. Preprocessing 1: Replacing Null/NaN data  {display-mode: "form" }

#@markdown <b> ---- Null Treatments </b>
Treatment_Choice_Numeric = "Replace Null with Zero" #@param ["Replace Null with Mean", "Replace Null with Max", "Replace Null with Min", "Replace Null with Zero", "Fill-in Null with Moving Average", "Delete rows with Null"]
Treatment_Choice_String = "Delete rows with Null" #@param ["Replace Null with most frequent", "Replace Null with empty string", "Delete rows with Null"]

#@markdown <b> ---- Drop columns above a %Null theshold </b>
Drop_Columns_Threshold = 30 #@param {type:"slider", min:0, max:100, step:10}

#@markdown <b> ---- Save changes to df. Run 1st time in "Preview Only" to check first, before running 2nd time in "Save to df"</b>
Options = "Save to df" #@param ["Preview only","Save to df", "Save to df and Export"]



def treat_nulls(df_, treatment_numeric, treatment_string, drop_threshold):

    # Drop Columns
    percentage = df_.isnull().sum()/df_.isnull().count()*100
    drop_list = list(percentage[percentage > drop_threshold].index)
    df_.drop(columns=drop_list, inplace=True)

    # Numeric treatments
    summary_tab = df_.describe()
    if treatment_numeric == 'Replace Null with Mean':
        for column, value in summary_tab.loc['mean', :].items():
            df_[column].fillna(value, inplace=True)
    elif treatment_numeric == 'Replace Null with Max':
        for column, value in summary_tab.loc['max', :].items():
            df_[column].fillna(value, inplace=True)
    elif treatment_numeric == 'Replace Null with Min':
        for column, value in summary_tab.loc['min', :].items():
            df_[column].fillna(value, inplace=True)
    elif treatment_numeric == 'Replace Null with Zero':
        for column, value in summary_tab.loc['max', :].items():
            df_[column].fillna(0, inplace=True)
    elif treatment_numeric == "Fill-in Null with Moving Average":
        for column, value in summary_tab.loc['max', :].items():
            df_[column].fillna(pd.rolling_mean(df_[column], 6, min_periods=1), inplace=True)
    elif treatment_numeric == "Delete rows with Null":
        df_ = df_.merge(df_[list(summary_tab.columns)].dropna(axis=0), left_index=True, right_index=True, how='inner', suffixes=('', '_y'))

    # String treatments
    strings_Cols = list(df_.select_dtypes(include='object').columns)
    if treatment_string == 'Replace Null with most frequent':
        for column in strings_Cols:
            df_[column].fillna(df_[column].value_counts().iloc[0:1].index[0], inplace=True)
    elif treatment_string == 'Replace Null with empty string':
        for column in strings_Cols:
            df_[column].fillna('', inplace=True)
    elif treatment_string == "Delete rows with Null":
        df_ = df_.merge(df_[strings_Cols].dropna(axis=0), left_index=True, right_index=True, how='inner', suffixes=('', '_y'))
    df_.drop([x for x in df_ if x.endswith('_y')], axis=1, inplace=True)
    return df_


tb = widgets.TabBar(["Null Counts", 'Before_Treatment', 'After_Treatment'], location='start')

with tb.output_to("Null Counts"):
    print('''-------------------------Null Counts--------------------------''')
    total_na_values = df.isnull().sum().sort_values(ascending=False)
    if total_na_values.sum() == 0:
        print('No Null Column! Yay!')
    else:
        percentage = round(df.isnull().sum()/df.isnull().count()
                           * 100, 2).sort_values(ascending=False)
        empty = pd.concat([total_na_values[total_na_values > 0], percentage[total_na_values > 0]], axis=1, keys=[
                          'Null in Counts', 'Null in Percents'])
        display(empty)

with tb.output_to('Before_Treatment'):
    print('''-------------------------Before Treatment--------------------------''')
    display(df.info())

with tb.output_to('After_Treatment'):
    print('''-------------------------After Treatment--------------------------''')
    df_null_treated = treat_nulls(df.copy(), Treatment_Choice_Numeric, Treatment_Choice_String, Drop_Columns_Threshold)
    display(df_null_treated.info())

if Options == "Save to df":
    if total_na_values.sum() > 0:
        df = df_null_treated
    print('--Changes are saved to df!--')
elif Options == "Save to df and Export":
    if total_na_values.sum() > 0:
        df = df_null_treated
    df.to_csv(dataset[:dataset.find('.')]+'_null_treated.csv')
    files.download(dataset[:dataset.find('.')]+'_null_treated.csv')
    print('--Download successful!--')
else:
    print("--Changes are only for preview, and not saved to df--")

del df_null_treated
gc.collect()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

-------------------------After Treatment--------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null int64
DebtRatio                               150000 non-null float64
MonthlyIncome                           150000 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null int64
NumberRealEstateLoansOrLines            150000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null int64
NumberOfDependents                      150000 non-null float64
dtypes: float64(4), int64(7)
memory usage: 12.6 MB


None

<IPython.core.display.Javascript object>

--Changes are saved to df!--


24

In [8]:
#@title ## 2.2. Preprocessing 2: Remove Features with High Correlation  {display-mode: "form" }
#@markdown Recursive Feature Elimination (RFE)
#@markdown  - adapted version that does not require target variables

#@markdown <b> ---- Define Correlation Threshold </b>
# Correlation_Max = '0.9' #@param ['0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1.0']
Correlation_Max = 0.9 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
df_corr = df.copy(deep=True)
max_corr = 1.0
dropped_cols = []

while max_corr > Correlation_Max:
    corr_rank = df_corr.corr().abs().unstack().sort_values(ascending=False)
    corr_rank = corr_rank[corr_rank < 1.0]
    high_corr = corr_rank[(corr_rank > Correlation_Max) & (corr_rank < 1.0)]
    max_corr = high_corr.max()
    if max_corr > Correlation_Max:
        drop_corr = high_corr.index[0][0]
        df_corr.drop(drop_corr, axis=1, inplace=True)
        dropped_cols.append(drop_corr)

df = df_corr

print(f'Dropped {len(dropped_cols)} Highly Correlated Features: \n{dropped_cols}')
print('\n')
print(f'Remaining Columns Features: \n{list(df_corr.columns)}')

Dropped 2 Highly Correlated Features: 
['NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTime30-59DaysPastDueNotWorse']


Remaining Columns Features: 
['SeriousDlqin2yrs', 'RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfDependents']


In [218]:
#@title ## 2.3. Preprocessing 3: Sampling (reduce computation) {display-mode: "form" }
#@markdown Takes a subsample of the entire dataset
Sample_Size = 10000 #@param {type:"integer"}

df_sample = df.sample(n=Sample_Size, random_state=42)

Options = "Save to df" #@param ["Preview only","Save to df", "Save to df and Export"]

if Options == "Save to df":
    df = df_sample
    print('--Changes are saved to df!--')
elif Options == "Save to df and Export":
    df = df_sample
    df.to_csv(dataset[:dataset.find('.')]+'_null_treated.csv')
    files.download(dataset[:dataset.find('.')]+'_null_treated.csv')
    print('--Download successful!--')
else:
    print("--Changes are only for preview, and not saved to df--")
    display(df_sample)

print('\nRemaining Columns')
print('-----------------------------')
pprint(list(df.columns))

--Changes are saved to df!--

Remaining Columns
-----------------------------
['SeriousDlqin2yrs',
 'RevolvingUtilizationOfUnsecuredLines',
 'age',
 'DebtRatio',
 'MonthlyIncome',
 'NumberOfOpenCreditLinesAndLoans',
 'NumberOfTimes90DaysLate',
 'NumberRealEstateLoansOrLines',
 'NumberOfDependents']


## 2.4. If You Want To Run Selected Columns Only
- add it into ```interested_cols```

In [0]:
###############################################
##  IF YOU WANT TO RUN SELECTED COLUMNS ONLY ##
###############################################
interested_cols = []
###############################################

if len(interested_cols) > 0:
    df = df[interested_cols]

variance_threhold_percent = 95

df_scaled = preprocessing.StandardScaler().fit_transform(df)
df_normed = pd.DataFrame(preprocessing.Normalizer().fit_transform(df_scaled), columns=df.columns)
original_cols = df_normed.columns
pca_ = PCA(n_components=variance_threhold_percent/100)
df_clustering = pd.DataFrame(pca_.fit_transform(df_normed[original_cols]))
new_cols = [f'PCA_{col}' for col in range(len(df_clustering.columns))]

# 3. Run Clustering

In [220]:
#@title ## 3.2. Choose metrics and algorithm to be run.

#@markdown __Clustering Algorithms__
KMedoid_Clustering = False #@param {type:"boolean"}
KMean_Clustering = True #@param {type:"boolean"}
Agglomerative_Clustering = True #@param {type:"boolean"}
Spectral_Clustering = False #@param {type:"boolean"}
DBSCAN_ = False #@param {type:"boolean"}

#@markdown __Evaluation Metrics__
Silhouette_Score = True #@param {type:"boolean"}
Davies_Bouldin_Score = True #@param {type:"boolean"}
Calinski_Harabasz_Score = True #@param {type:"boolean"}

# USER CONFIG
metrics_list = [
                'Silhouette_Score',
                'Davies_Bouldin_Score',
                'Calinski_Harabasz_Score',
                ]

algorithm_list = [
                'KMedoid_Clustering',
                'KMean_Clustering',
                'Agglomerative_Clustering',
                'Spectral_Clustering',
                'DBSCAN_'
                ]

metrics_list = [metrics for metrics in metrics_list if eval(f'{metrics} == True')]
algorithm_list = [algo.replace('_','') for algo in algorithm_list if eval(f'{algo} == True')]

max_clusters = 100
clustering_list = []
Clustering_results = pd.DataFrame()
eps = optimal_eps(df_normed.to_numpy())

def optimal_eps(X):
    nbrs = NearestNeighbors(n_neighbors=2)
    nbrsfit = nbrs.fit(X)
    distances, indices = nbrsfit.kneighbors(X)
    distances = np.sort(distances, axis=0)[:,1]
    test = pd.DataFrame(distances, columns=['Y2'])
    test['Interval'] = np.where(test.index % int(len(test)/50) == 0, 1, 0)
    smaller_test = test[test.Interval == 1]
    smaller_test['Grad'] = (smaller_test['Y2'] - smaller_test.Y2.shift(1)).fillna(0)
    smaller_test['Grad2'] = (smaller_test['Grad'] - smaller_test.Grad.shift(1)).fillna(0)
    smaller_test['Grad3'] = smaller_test['Grad2'].rolling(int(len(smaller_test)/10)).std()
    smaller_test['Grad4'] = np.where(smaller_test['Grad'] > (smaller_test['Grad'].shift(1) + abs(smaller_test['Grad3'].shift(1)*3)) , 1, 0)
    return np.round(smaller_test[smaller_test['Grad4'] == 1 ].min()['Y2'], 1)

def get_metrics(X, labels, metrics_list_):
    results_list_ = []
    
    for metric in metrics_list_:
        if eval(f'{metric} == True'):
            try:
                result = eval(f'{metric.lower()}(X, labels)')
            except:
                result = np.nan
            results_list_.append(result)
    return results_list_

def KMedoidClustering(X, kmin=2, n_clusters=11):
    from pyclustering.cluster.kmedoids import kmedoids
    simi_matrix = 1 - np.dot(X, X.T)
    kmedoids_instance = kmedoids(simi_matrix, [kmin, n_clusters], data_type='distance_matrix')
    kmedoids_instance.process()
    clusters = kmedoids_instance.get_clusters()
    return {v_: i for i, v in enumerate(clusters) for v_ in v}
    
def KMeanClustering(X, kmin=2, n_clusters=11):
    elbow_instance = elbow(X, kmin, n_clusters)
    amount_clusters = elbow_instance.process().get_amount()
    centers = kmeans_plusplus_initializer(X, amount_clusters, kmeans_plusplus_initializer.FARTHEST_CENTER_CANDIDATE).initialize()
    kmeans_instance = kmeans(X, centers).process()
    clusters = kmeans_instance.get_clusters()
    return {v_: i for i, v in enumerate(clusters) for v_ in v}

for algo in algorithm_list:
    if 'AgglomerativeClustering' in algo:
        for i in np.arange(0, max_clusters, 5):
            if i == 0: i = 2
            cluster = AgglomerativeClustering(n_clusters=i, affinity='cosine', linkage='complete')
            cluster.fit_predict(df_normed.to_numpy())
            metrics =  get_metrics(df_normed.to_numpy(), cluster.labels_, metrics_list)
            clustering_list.append([algo] + [np.nan] + [i] + metrics)
    elif 'DBSCAN' in algo:
        for samples in np.arange(0, 30, 5):
            cluster = DBSCAN(eps=eps, min_samples = samples)
            cluster.fit_predict(df_normed.to_numpy())
            metrics = get_metrics(df_normed.to_numpy(), cluster.labels_, metrics_list)
            clustering_list.append([algo] + [samples] + [pd.Series(cluster.labels_).nunique()] + metrics)
    elif 'SpectralClustering' in algo:
        for i in np.arange(0, max_clusters, 5):
            if i == 0: i = 2
            cluster = SpectralClustering(n_clusters=i, assign_labels="discretize", random_state=0)
            cluster.fit_predict(df_normed.to_numpy())
            metrics =  get_metrics(df_normed.to_numpy(), cluster.labels_, metrics_list)
            clustering_list.append([algo] + [np.nan] + [i] + metrics)
    else:
        cluster = eval(f'{algo}(df_normed.to_numpy(), n_clusters=max_clusters)')
        metrics =  get_metrics(df_normed.to_numpy(), pd.Series(cluster).to_numpy(), metrics_list)
        no_clusters = pd.Series(cluster).nunique()
        clustering_list.append([algo]  + [np.nan] + [no_clusters] + metrics)

display(pd.DataFrame(clustering_list, index=None, columns=['Algorithm', 'SampleSize','ClusterCount']+metrics_list))
# cluster = _KMedoidClustering(df_normed.to_numpy(), 1, 100)


,Algorithm,SampleSize,ClusterCount,Silhouette_Score,Davies_Bouldin_Score,Calinski_Harabasz_Score
0,KMeanClustering,NaN,16,-0.053275,72.756173,0.963830
1,AgglomerativeClustering,NaN,2,0.180001,2.132888,2093.524551
2,AgglomerativeClustering,NaN,5,0.179043,1.758362,1911.554506
3,AgglomerativeClustering,NaN,10,0.174360,1.794178,1474.647142
4,AgglomerativeClustering,NaN,15,0.162836,1.738900,1384.742985
5,AgglomerativeClustering,NaN,20,0.172373,1.578076,1232.550929
6,AgglomerativeClustering,NaN,25,0.163089,1.545102,1085.040297
7,AgglomerativeClustering,NaN,30,0.139131,1.570824,1005.060900
8,AgglomerativeClustering,NaN,35,0.145286,1.522241,948.794094
9,AgglomerativeClustering,NaN,40,0.142998,1.498211,892.387945
